# Install & setup library
To install inferscope pythoh library just run `pip install inferscope`
Then please visit [token page https://app.inferscope.tech/token](https://app.inferscope.tech/token) token page, and follow instructions for local token setup. Inferscope library reads token from 3 different places:
* `~/.inferscope/token` file
* `INFERSCOPE_TOKEN` environment variable
* By passing it directly in client constructor
```python
Client(token='IS-Tok-XXXX')
```

  

In [1]:
#uncomment to install
#!pip install inferscope 

In [2]:
from IPython.display import display, Markdown, Latex

In [3]:
import inferscope as infs
import pandas as pd
import numpy as np
import os
from uuid import UUID

In [4]:
from inferscope.models.data_description import (ColumnInformation, DataDescription, DataFormat, DataFormatType,
                                                DataType, ImageSematicProperties, JoinType, SemanticType)
from inferscope.models import ModelInfo, DatasetInfo, Metric, MetricBestValue

**Now we have everything needed for this demo imported and ready to run
Let's create `client` and, if needed, setup `parent_project_id` if specific artifacts/runs location needed**

In [5]:
client = infs.Client()
parent_project_id=None # UUID("Your project id, if not set, default project will be used")


# Adding simple run without artifacts
In following cell you can add simple run just to check everything works

In [6]:
def pretty_print_run(run: infs.Run):
    # you can look at raw data
    # print(added_run) 
    run_link = f"https://app.inferscope.tech/run/{run.uid}"
    display(Markdown(f'''You can now look at added run in UI by this [link {run_link}]({run_link})'''))

In [7]:
run = infs.Run(
    # You can provide your meaningful run name, or just leave it empty
    # name=None,
    name="Example run without artifacts",
    metrics=[
        Metric(name='avg_time', value=np.random.randn() * 3 + 7, best_value="min"),
        Metric(name='quality_score', value=np.random.randn() * 10, best_value="max"),
        Metric(name='quality_score', value=np.random.randn() * 10, slice="person", best_value="max"),
        Metric(name='quality_score', value=np.random.randn() * 10, slice="animals", best_value="max"),
        
    ],
    model=ModelInfo(
        name="some model",
        version="0.0.1",
        description="Some model produced or used in run. "
        "We plan to add more sophisticated schema (produced model(s), consumed model(s)) later, if needed - contact the team."
    ),
    dataset=DatasetInfo(
        name="some dataset",
        version="any string can be a version",
        description="Some dataset produced or used in this run. "
        "We plan to add more sophisticated schema (produced model(s), consumed model(s)) later, if needed - contact the team."
    ),
    parent_project_uid=parent_project_id,
    description='''# Markdown description
Example run description. Can contain all markdown elements such as [links](https://inferscope.tech) and other.
    '''
)
added_run = client.add(run)
pretty_print_run(added_run)

You can now look at added run in UI by this [link https://app.inferscope.tech/run/742d7d58-fd5a-43a9-977b-e46d582bddf4](https://app.inferscope.tech/run/742d7d58-fd5a-43a9-977b-e46d582bddf4)

# Now adding our first run with artifact - externally store dataframe with links to publicly available content
We prepared some publicly available datasets in our s3 bucket by this links:
[https://storage.googleapis.com/inferscope_example_data/left_img_dataset.tsv](https://storage.googleapis.com/inferscope_example_data/left_img_dataset.tsv)
[https://storage.googleapis.com/inferscope_example_data/right_img_dataset.tsv](https://storage.googleapis.com/inferscope_example_data/right_img_dataset.tsv)

Those datasets contain 4 columns - `ids`, `prompts`, `generation_time`, `img_urls`

First, we need to define data schema for this dataframe

In [8]:
data_description = DataDescription(
    data_format=DataFormat(
        dsv_delimiter='\t', # separator, can be any separator, behavior like in pandas.read_csv
        data_format=DataFormatType.DSV # we support DSV, delimiter separated format or Json in each line (aka JSONEachRow)
    ),
    columns=[
        ColumnInformation(name="ids", type="integer"),
        ColumnInformation(name="generation_time", type="float", semantic="metric"),
        ColumnInformation(name="img_urls", type="url", semantic="image"),
        ColumnInformation(name="prompts", type="string"),
    ],
)

When working with external links, it's possible to skip ArtifactPack creation and pass list of `ExternalLinkArtifact` directly in `Run` constructor.

In [9]:
run = infs.Run(
    artifacts=[
        infs.ExternalLinkArtifact(
            data_description=data_description,
            path="artifact",
            uri=f"https://storage.googleapis.com/inferscope_example_data/left_img_dataset.tsv",
            semantic="dataframe",
        )
    ],
    dataset=DatasetInfo(name=f"example dataset for ext_links", version="0.0.1"),
    metrics=[
        Metric(name="some_metric_name", value=0.8),
    ],
    name=f'Example run with ext link artifacts'
)
added_run = client.add(run)
pretty_print_run(added_run)
print("Please open 'Dataframe view' tap in ui to see the data")

You can now look at added run in UI by this [link https://app.inferscope.tech/run/09e4260f-acd2-4a59-b3d4-7cb3d5c01b17](https://app.inferscope.tech/run/09e4260f-acd2-4a59-b3d4-7cb3d5c01b17)

Please open 'Dataframe view' tap in ui to see the data


# Uploading artifacts to inferscope internal storage
We also support storing datarfames and content in our storage. We have `infs.StoredArtifactHelper` for uploading data and getting special links `iss://` that our UI & Python library can recognize and show/download their content.
We generated some images with bing img gen and saved them in `generated_imgs` folder

In [10]:
!ls generated_imgs

adventurous-fox-knight-in-armor--a-dashing-cartoon  _right.jpg
adventurous-fox-knight-in-armor--a-dashing-cartoon.jpg
autumn-festival-in-a-fantasy-village--a-village-ce  _right.jpg
autumn-festival-in-a-fantasy-village--a-village-ce.jpg
elderly-fisherman-by-the-seaside--a-close-up-portr  _right.jpg
elderly-fisherman-by-the-seaside--a-close-up-portr.jpg
ethereal-underwater-market--a-bustling-underwater-  _right.jpg
ethereal-underwater-market--a-bustling-underwater-.jpg
floating-island-castle--a-majestic-castle-rests-at  _right.jpg
floating-island-castle--a-majestic-castle-rests-at.jpg
misty-alpine-forest-at-dawn--a-serene-landscape-of  _right.jpg
misty-alpine-forest-at-dawn--a-serene-landscape-of.jpg
panda-owl-in-a-bowl-  _right.jpg
panda-owl-in-a-bowl-.jpg
portrait-of-a-young-woman-with-freckles--a-close-u  _right.jpg
portrait-of-a-young-woman-with-freckles--a-close-u.jpg
portrait-of-a-young-woman-with-freckles--a-close-u.png
rainbow-waterfall-with-magical-creatures--a-colorf  _right.jp

Let's upload one of those images and use it in dataframe with it's prompt (extracted from filename)

In [11]:
local_img_path = "generated_imgs/adventurous-fox-knight-in-armor--a-dashing-cartoon.jpg"
prompt = "adventurous fox knight in armor a dashing cartoon"
simple_artifact_pack = infs.StoredArtifactHelper(
    client, parent_project_id
)
uploaded_img = simple_artifact_pack.upload_artifact(
    "adventurous-fox-knight-in-armor--a-dashing-cartoon.jpg", 
    local_path=local_img_path,
    data_description=DataDescription(
        # You need to provide data_format for DataDescription instantiation
        data_format=DataFormat(data_format=DataFormatType.Binary,),
        # we suggest that we have image in our file, that's not needed but it'll be usefull in future
        semantic="image",
    )
)
simpe_df = pd.DataFrame([{
    "ids": 1,
    "img_urls": uploaded_img.uri, # we just put our iss:// url so artifact or diff viewer can display it properly
    "prompts": prompt,
    "generation_time": 1.381
}])
# upload_artifact can work with binary and regual strings, so let's try it
simple_artifact_pack.upload_artifact(
    path="dataframe",
    blob=simpe_df.to_csv(sep='\t',index=None),
    data_description=data_description
)
run = infs.Run(
    name="run with single image",
    artifact_pack_id=simple_artifact_pack.pack_uid,
)
added_run = client.add(run)
pretty_print_run(added_run)

You can now look at added run in UI by this [link https://app.inferscope.tech/run/7769f2e2-fbab-4507-81d6-38c856b7e6ee](https://app.inferscope.tech/run/7769f2e2-fbab-4507-81d6-38c856b7e6ee)

## Now to the most important example - multi lines example, ready for diffing!

In [14]:
def gen(right=False):
    package = infs.StoredArtifactHelper(
        client, parent_project_id
    )
    names = sorted(list(filter(lambda x: x.endswith('jpg') and (('_right' in x) == right),os.listdir('generated_imgs'))))
    d = pd.DataFrame()
    d['ids'] = [i for i in range(len(names))]
    d['prompts'] = [' '.join(name[:-4].split('-')) for name in names]
    d['generation_time'] = np.random.randn(len(names)) * 3 + 7
    d['img_urls'] = [
        package.upload_artifact(
            name, 
            local_path=f'generated_imgs/{name}',
            data_description=DataDescription(
                    data_format=DataFormat(data_format=DataFormatType.Binary,), semantic="image",
                ),).uri for name in names]
    package.upload_artifact(
        blob=d.to_csv(sep='\t',index=None), 
        path='dataframe',
        data_description=data_description,
    )
        
    run = infs.Run(
        artifact_pack_id=package.pack_uid,
        metrics=[
            Metric(name='avg_time', value=np.random.randn() * 3 + 7, best_value="min"),
            Metric(name='quality_score', value=np.random.randn() * 10, best_value="max"),
            Metric(name='quality_score', value=np.random.randn() * 10, slice="person", best_value="max"),
            Metric(name='quality_score', value=np.random.randn() * 10, slice="animals", best_value="max"),
            
        ],
        parent_project_uid=parent_project_id,
        description='''#Markdown description
Example run description. Can contain all markdown elements such as [links](https://inferscope.tech), lists
        '''
    )
    return client.add(run)
right_run = gen(True)
pretty_print_run(right_run)
left_run = gen(False)
pretty_print_run(left_run)
direct_diff_url = f'https://app.inferscope.tech/diff_view?run_uids={left_run.uid},{right_run.uid}'
display(Markdown(f'''
Now you can visit UI, or just this direct url [{direct_diff_url}]({direct_diff_url})
'''
))

You can now look at added run in UI by this [link https://app.inferscope.tech/run/2e746330-406f-410b-be67-b24883d06cc5](https://app.inferscope.tech/run/2e746330-406f-410b-be67-b24883d06cc5)

You can now look at added run in UI by this [link https://app.inferscope.tech/run/6f4b0420-8014-4765-8a6b-dafbb8a1193b](https://app.inferscope.tech/run/6f4b0420-8014-4765-8a6b-dafbb8a1193b)


Now you can visit UI, or just this direct url [https://app.inferscope.tech/diff_view?run_uids=6f4b0420-8014-4765-8a6b-dafbb8a1193b,2e746330-406f-410b-be67-b24883d06cc5](https://app.inferscope.tech/diff_view?run_uids=6f4b0420-8014-4765-8a6b-dafbb8a1193b,2e746330-406f-410b-be67-b24883d06cc5)
